# Big Data Project

## Configuration

In [1]:
%%configure -f
{"executorMemory":"8G", "numExecutors":2, "executorCores":3, "conf": {"spark.dynamicAllocation.enabled": "false"}}

In [22]:
//Denys: val bucketname = "unibo-bd2122-dgrushchak"
//Riccardo: val bucketname = "unibo-bd2223-rbacca"

val bucketname = "unibo-bd2122-dgrushchak"

//Denys: val path_flights_db = "s3a://"+bucketname+"/project/small_file_10000000.txt"
//Riccardo:val path_flights_db = "s3a://"+bucketname+"/bigdata-project/part_1.txt"

val path_flights_db = "s3a://"+bucketname+"/projectTest/xaa.txt"

sc.applicationId

"SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/" + sc.applicationId + "/"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

bucketname: String = unibo-bd2122-dgrushchak
path_flights_db: String = s3a://unibo-bd2122-dgrushchak/projectTest/xaa.txt
res71: String = application_1674753577028_0003
res73: String = SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/application_1674753577028_0003/


In [32]:
object Parser{
    val commaRegex = ",(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)"
    val doubleVerticalLine = "\\|\\|"
    
    val pipeRegex = "\\|(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)" // not used
    val quotes = "\"" //not used
   
    
    case class FlightData(
        flightDate: String,          //1
        startingAirport: String,     //2
        destinationAirport: String,  //3
        travelDuration: Int,         //4
        airplaneType: Array[String], //5
        airlineName: Array[String],  //6
        isNonStop: Boolean,          //7
        isBasicEconomy: Boolean,     //8
        totalFare: Double,           //9
        segmentsDuration: Array[Int],//10
        segmentsDistance: Array[Int] //11
    ){
        def un() = FlightData.unapply(this).get
    }
    
    def convertTravelDuration(value: String): Int = {
        val input = value.replace("PT", "")
        val minutes = input.replace("M", "").split("H") //[2, 35]
        
        (minutes(0).trim.toInt)*60 + minutes(1).trim.toInt
    }
    
    def convertAirplaneType(value: String): Array[String] = {
        //Ex. Airbus A320||Boeing 737-800
        val result = value.split(doubleVerticalLine)
        if (result.head == "") Array.empty
        else result
    }
    
    def convertAirlineName(value: String): Array[String] = {
        val result = value.split(doubleVerticalLine)
        if (result.head == "") Array.empty
        else result
    }
    
    //return array of duration of each leg in minutes
    def convertSegmentsDuration(value: String): Array[Int] = {
        val result = value.split(doubleVerticalLine)
        if (result.head == "") Array.empty
        else result.map(_.toInt/60)
    }
    
    def convertSegmentsDistance(value: String): Array[Int] = {
        val result = value.split(doubleVerticalLine)
        if(result.head == "" || result.head == "None") Array.empty
        else result.map(_.toInt)
    }
    
     /**
        2 - flightdate (String) ** x._1
        3 - starting airport (String) ** x._2
        4 - destination airport (String) ** x._3
        6 - travel duration (PT2H35M) ** in minutes x._4
        8 - isBasicEconomy (String) ** x._8
        9 - isrefundable (boolean)
        10 - isNonStop (String) ** x._7
        12 - basefare (prezzo base del ticket in $ Double) ** x._9
        12 - seats remaining (Int)
        20 - airline name (company name) ** x._6
        23 - airplane type (Vector of [String||String] ** x._5
        25 - segment distance (distance in milles: Vector of [Int||Int]  can be [None||None]) ** x._11
        24 - segments duration in seconds x._10
    **/
    
     def parseFlightInformationLine(line: String): Option[FlightData] = {
        try {
            val input = line.split(commaRegex)
            
            val airplanes = convertAirplaneType(input(23))
            val airline = convertAirlineName(input(21))
            val segmentsDuration = convertSegmentsDuration(input(24))
            val segmentsDistance = convertSegmentsDistance(input(25))

            if (airplanes.isEmpty || airline.isEmpty || segmentsDuration.isEmpty || segmentsDistance.isEmpty) None
            else Some(FlightData(
                input(2), 
                input(3), 
                input(4), 
                convertTravelDuration(input(6)), 
                airplanes, 
                airline, 
                input(10).toBoolean, 
                input(8).toBoolean, 
                input(12).toDouble, 
                segmentsDuration,
                segmentsDistance))
        } catch {
            case _: Exception => None
        }
    }
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

defined object Parser


In [33]:
sc.getPersistentRDDs.foreach(_._2.unpersist()) 
val rddFlights = sc.textFile(path_flights_db).flatMap(Parser.parseFlightInformationLine)
val rddFlightsCached = rddFlights.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

rddFlights: org.apache.spark.rdd.RDD[Parser.FlightData] = MapPartitionsRDD[97] at flatMap at <console>:25
rddFlightsCached: rddFlights.type = MapPartitionsRDD[97] at flatMap at <console>:25


In [35]:
rddFlightsCached.map(_.un()).take(10).foreach(println)


def round(v: Double): Double = {
    (v*100).toInt/100.toDouble
}

val numberOfRecords = rddFlightsCached.count


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(2022-04-17,ATL,BOS,149,[Ljava.lang.String;@74dd7776,[Ljava.lang.String;@4850116d,true,false,248.6,[I@47f48f1f,[I@16627996)
(2022-04-17,ATL,BOS,150,[Ljava.lang.String;@99bcaeb,[Ljava.lang.String;@410869cc,true,false,248.6,[I@22679a85,[I@274ce996)
(2022-04-17,ATL,BOS,150,[Ljava.lang.String;@7057b306,[Ljava.lang.String;@7213e2c9,true,false,248.6,[I@6c3efdbc,[I@2c7ea9f4)
(2022-04-17,ATL,BOS,152,[Ljava.lang.String;@61fce0fe,[Ljava.lang.String;@c9443a2,true,false,248.6,[I@17dc8c91,[I@4e34a697)
(2022-04-17,ATL,BOS,154,[Ljava.lang.String;@484944bf,[Ljava.lang.String;@666e8747,true,false,248.6,[I@15a5371f,[I@7e4e2b22)
(2022-04-17,ATL,BOS,252,[Ljava.lang.String;@7ca0e4e8,[Ljava.lang.String;@6ee60084,false,false,251.1,[I@52c7df88,[I@5d5ff9c9)
(2022-04-17,ATL,BOS,318,[Ljava.lang.String;@26a3c12e,[Ljava.lang.String;@781f175e,false,false,251.1,[I@1448ac4a,[I@36fb9ebb)
(2022-04-17,ATL,BOS,332,[Ljava.lang.String;@ee993fb,[Ljava.lang.String;@e4cd09f,false,false,251.1,[I@5fb707e0,[I@35d354eb)
(2022-04-

## Explorative queries

1. How many distinct airports and aircraft models
2. Average travel duration for airline, only in non-stop flights
3. Percentage of basic economy tickets, based on all tickets
4. Percentage of non-stop flights (flights with one leg)
5. Average and price range of tickets
6. Average ticket price for each airline
7. Average and range of travel distance
8. Top 10 airports with more arriving flights


In [36]:
//1.How many distinct airports and aircraft models
val distinctAirports = rddFlightsCached.
    map(x => x.startingAirport).
    distinct.union(
        rddFlightsCached.map(x => x.destinationAirport).distinct).
    distinct.count
val distinctAircraftModels = rddFlightsCached.flatMap(x => x.airplaneType).distinct.count

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

distinctAirports: Long = 16
distinctAircraftModels: Long = 42


In [37]:
//2. Average travel duration for each airline, in non stop flights
val distinctAirlines = rddFlightsCached.
    filter(x => x.isNonStop).
    map(x => (x.airlineName(0), x.travelDuration)).
    aggregateByKey((0.0, 0.0))((a,v)=>(a._1+v, a._2+1), (a1,a2)=>(a1._1+a2._1, a1._2+a2._2)).
    map({case(k,v)=>(k,v._1/v._2)}).
    collect().
    foreach({case (airline, value) => println(airline + " => " + round(value) + " avg minutes")})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Frontier Airlines => 165.84 avg minutes
United => 186.45 avg minutes
Delta => 152.53 avg minutes
Alaska Airlines => 243.72 avg minutes
American Airlines => 183.45 avg minutes
JetBlue Airways => 200.58 avg minutes
distinctAirlines: Unit = ()


In [38]:
//3. Percentage of basic economy tickets, based on all tickets
round((rddFlightsCached.filter(_.isBasicEconomy).count.toDouble/numberOfRecords).toDouble*100) + " %"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res109: String = 8.76 %


In [39]:
//4. Percentage of non-stop flights (flights with one leg)
round((rddFlightsCached.filter(_.isNonStop).count.toDouble/numberOfRecords).toDouble*100) + " %"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res111: String = 28.93 %


In [40]:
//5. Average and price range of tickets
val ticketPrices = rddFlightsCached.map(x => x.totalFare)
"Range of prices: "  + ticketPrices.min + " to " + ticketPrices.max
"Avg price: " + round((ticketPrices.sum/numberOfRecords).toDouble)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ticketPrices: org.apache.spark.rdd.RDD[Double] = MapPartitionsRDD[123] at map at <console>:24
res113: String = Range of prices: 23.97 to 3810.6
res114: String = Avg price: 357.51


In [41]:
//6. Average ticket price for each airline
val avgTicketPricePerAirline = rddFlightsCached.
    filter(_.isNonStop).
    map(x => (x.airlineName(0),(x.totalFare, 1))).
    reduceByKey((a,b) => (a._1+b._1, a._2+b._2)).
    map(m => (m._1,m._2._1/m._2._2)).
    collect.foreach({case (name, value) => println(name + " => " + round(value))})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Frontier Airlines => 112.7
United => 283.57
Delta => 249.71
Alaska Airlines => 277.86
American Airlines => 267.75
JetBlue Airways => 231.55
avgTicketPricePerAirline: Unit = ()


In [ ]:
//7. Average and range of travel distance

In [42]:
//8. Top 10 airports with more arriving flights
val topAirports = rddFlightsCached.
    map(x => (x.destinationAirport, 1)).
    reduceByKey(_+_).
    sortBy(_._2, false).
    take(10).
    foreach({case (name, value) => println(name + " => " + value)})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

LAX => 40881
LGA => 35280
MIA => 30886
BOS => 30416
SFO => 30048
ORD => 29603
DFW => 29516
CLT => 29076
DEN => 27537
ATL => 27403
topAirports: Unit = ()


## Main queries


In [ ]:
val path_output = "s3a://"+bucketname+"/spark/avgRatPerMovie" //todo

### Riccardo

> Riccardo: aggrego su “aircraft models” per calcolare la classifica discreta dei modelli più usati rispetto a ciascuna “airline”. Eseguo il join col dataset originale, infine riaggrego su “travel duration” e sulla classificazione di prima. Ottengo la durata di media di ogni volo per ogni compagnia (oltre a numero di voli e totale di ore) sul modello di aereo più utilizzato

In [43]:
// Query approfondita --> Riccardo

val zippedRdd = rddFlightsCached.map(x => (x.airplaneType, x.airlineName)).zipWithIndex()
val aircraft = zippedRdd.map(x => (x._2, x._1._1.zipWithIndex)).flatMapValues(x => x).map(x => ((x._1, x._2._2), x._2._1))
val airlines = zippedRdd.map(x => (x._2, x._1._2.zipWithIndex)).flatMapValues(x => x).map(x => ((x._1, x._2._2), x._2._1)) 

val mostUsedModelForAirline = aircraft.join(airlines).map(x => ((x._2._2, x._2._1), 1)).reduceByKey(_+_).sortBy(s => (s._1._1, s._2), false).take(10).foreach(println)

// val joinedWithOriginal = mostUsedModelForAirline.join(rddFlightsCached)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

zippedRdd: org.apache.spark.rdd.RDD[((Array[String], Array[String]), Long)] = ZippedWithIndexRDD[136] at zipWithIndex at <console>:25
aircraft: org.apache.spark.rdd.RDD[((Long, Int), String)] = MapPartitionsRDD[139] at map at <console>:23
airlines: org.apache.spark.rdd.RDD[((Long, Int), String)] = MapPartitionsRDD[142] at map at <console>:23
((United,Embraer 175 (Enhanced Winglets)),37860)
((United,Boeing 737-800),34429)
((United,Boeing 737-900),34098)
((United,Airbus A319),29032)
((United,Airbus A320),25233)
((United,Embraer 170),9312)
((United,Boeing 737 MAX 9),9053)
((United,Boeing 737-700),8019)
((United,Boeing 757-200),5287)
((United,Boeing 757-300),4799)
mostUsedModelForAirline: Unit = ()


### Denys
> Denys: aggrego su “aircraft model” per calcolare la “segment distance” totale percorsa da ogni modello, poi faccio self-join e aggregazione per determinare il “travel duration” per ogni “aircraft model”. Alla fine determino la velocità di ogni modello partendo dai dati aggregati.

In [45]:
rddFlightsCached.map(x => (x.airplaneType, x.segmentsDistance)).zipWithIndex.take(10)
rddFlightsCached.map(x => (x.airplaneType, x.segmentsDuration)).zipWithIndex.take(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res124: Array[((Array[String], Array[Int]), Long)] = Array(((Array(Airbus A321),Array(947)),0), ((Array(Airbus A321),Array(947)),1), ((Array(Boeing 757-200),Array(947)),2), ((Array(Airbus A321),Array(947)),3), ((Array(Airbus A321),Array(947)),4), ((Array(Airbus A320, Airbus A320),Array(228, 728)),5), ((Array(Airbus A320, Boeing 737-800),Array(228, 728)),6), ((Array(Airbus A319, Airbus A320),Array(228, 728)),7), ((Array(Airbus A319, Boeing 737-800),Array(228, 728)),8), ((Array(Airbus A319, Boeing 737-800),Array(541, 406)),9))
res126: Array[((Array[String], Array[Int]), Long)] = Array(((Array(Airbus A321),Array(149)),0), ((Array(Airbus A321),Array(150)),1), ((Array(Boeing 757-200),Array(150)),2), ((Array(Airbus A321),Array(152)),3), ((Array(Airbus A321),Array(154)),4), ((Array(Airbus A320, Airbus A320),Array(90, 125)),5), ((Array(Airbus A320, Boeing 737-800),Array(90, 138)),6), ((Array(Airbus A319, Airbus A320),Array(81, 125)),7), ((Array(Airbus A319, Boeing 737-800),Array(81, 138)),8), 